# Connect 4

In [1]:
import itertools
import numpy as np
import pandas as pd

In [2]:
from platform import python_version
python_version()

'3.6.10'

In [3]:
from connectx import Connect4

In [4]:
game = Connect4()
game.name

'Connect 4'

In [5]:
game.make_move((1, 0))
game.show_state()

_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
X _ _ _ _ _ _


In [6]:
game.make_move((2, 4))
game.show_state()

_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
X _ _ _ O _ _


In [7]:
while True:
    game = Connect4()
    while not game.game_over:
        role = game.turn
        moves = game.available_moves()
        move = np.random.choice(moves)
        game.make_move((role, move))
    if game.winner is None:
        break

game.show_state()
print(game.winner)

X O O O X O X
X X O X X X O
O X X O O O X
X O O O X O X
O O X X O O X
X X X O O X O
None


In [8]:
game.moves

[(1, 2),
 (2, 6),
 (1, 2),
 (2, 4),
 (1, 0),
 (2, 2),
 (1, 1),
 (2, 4),
 (1, 4),
 (2, 0),
 (1, 0),
 (2, 4),
 (1, 6),
 (2, 3),
 (1, 6),
 (2, 1),
 (1, 2),
 (2, 2),
 (1, 6),
 (2, 1),
 (1, 4),
 (2, 2),
 (1, 5),
 (2, 6),
 (1, 6),
 (2, 5),
 (1, 3),
 (2, 3),
 (1, 1),
 (2, 0),
 (1, 4),
 (2, 5),
 (1, 0),
 (2, 3),
 (1, 3),
 (2, 5),
 (1, 5),
 (2, 5),
 (1, 0),
 (2, 3),
 (1, 1),
 (2, 1)]

### Test check_game_state methods

In [9]:
moves = [
            (1, 3), (2, 6), (1, 0), (2, 5), (1, 2),
            (2, 2), (1, 5), (2, 1), (1, 3), (2, 1),
            (1, 6), (2, 2), (1, 4), (2, 4), (1, 4),
            (2, 4), (1, 1), (2, 3)
        ]
game = Connect4(moves=moves)
game.show_state()

_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ O _ _
_ X O O X _ _
_ O O X O X X
X O X X X O O


In [10]:
game.check_game_state()

(True, 2)

In [11]:
%timeit game.check_game_state()

40.6 µs ± 604 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [12]:
# Game execution speed test
%timeit Connect4(moves=moves)

788 µs ± 5.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
# 3.06 ms with both checks
# 2.18 ms with full-state check
# 1.04 ms with checks on last move's position
# 807 µs with checks on last move's position and fast check for draw

In [14]:
# Test game with standard players
from gamelearner import GameController, RandomPlayer, GameController

game = Connect4()
player1 = RandomPlayer('R1')
player2 = RandomPlayer('R2')

In [15]:
players = [player1, player2]
ctrl = GameController(game, players)

In [16]:
ctrl.play(show=True)

Game of Connect 4 with 2 players ['R1', 'R2']
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
R1's turn (column from left): 4
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ X _ _
R2's turn (column from left): 5
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ X O _
R1's turn (column from left): 1
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ X _ _ X O _
R2's turn (column from left): 1
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ O _ _ _ _ _
_ X _ _ X O _
R1's turn (column from left): 5
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ O _ _ _ X _
_ X _ _ X O _
R2's turn (column from left): 0
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ O _ _ _ X _
O X _ _ X O _
R1's turn (column from left): 0
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
X O _ _ _ X _
O X _ _ X O _
R2's turn (column from left): 6
_ _ _ _ _ _ _
_ _ _ _ _ _ 

In [17]:
# Find winning moves from current state

def _wins_from_next_move(game, role, board_full=None):
    if board_full is None:
        board_full = game.board_full
        state = game.state
    else:
        state = game.state_from_board_full(board_full)        
    wins = {}
    fill_levels = game._get_fill_levels(state)
    for col in game.available_moves(state):
        position = (fill_levels[col], col)
        pos_fb = position[0]+1, position[1]+1
        win = game._check_game_state_from_position(pos_fb, role, board_full=board_full)
        wins[col] = win
    return wins

def check_for_obvious_move(game, role, board_full=None):
    """Analyses the current board state (or board_full if
    provided) from the perspective of the player role.
    
    Returns
        value, positions (float, list): value of the current
            state if it is a terminal state (1.0, 0.5 or -1.0, 
            else None), a list of best positions (columns) to 
            take in next move.
    """
    if board_full is None:
        board_full = game._board_full
        state = game.state
        fill_levels = game._fill_levels
    else:
        state = game.state_from_board_full(board_full)
        fill_levels = game._get_fill_levels(state)
    opponent = role ^ 3
    n_moves = fill_levels.sum()

    # 0. Check if early move of game
    if n_moves == 0:
        return None, [3]
    elif (n_moves == 1) and (state[0,3] == opponent):
        return None, [3]
    
    # 1. Check for a win by role on next move
    wins = _wins_from_next_move(game, role, board_full=board_full)
    n_wins = sum(wins.values())
    if n_wins > 0:
        return 1.0, [col for col, win in wins.items() if win]

    # 2. Check if draw (last move but no win)
    if len(wins) == 1:
        return 0.5, list(wins.keys())

    # 3. Check which moves will lead to possible opponent win on next move
    bf2 = board_full.copy()
    state = game.state_from_board_full(bf2)
    fill_levels = game._get_fill_levels(state)
    opp_n_wins = {}
    for col in wins:
        assert state[fill_levels[col], col] == 0
        state[fill_levels[col], col] = role
        #import pdb; pdb.set_trace()
        opponent_wins = _wins_from_next_move(game, opponent, board_full=bf2)
        state[fill_levels[col], col] = 0
        opp_n_wins[col] = sum(opponent_wins.values())
    opp_wins = {c: n_wins > 0 for c, n_wins in opp_n_wins.items()}

    # 4. Check if avoiding possible opponent win is impossible
    if all(opp_wins.values()):
        lowest = [col for col, n_wins in opp_n_wins.items() 
                  if n_wins == min(opp_n_wins.values())]
        return -1.0, lowest

    # 5. If any move avoids a possible opponent win, take it
    if sum(opp_wins.values()) == len(opp_wins) - 1:
        return None, [col for col, win in opp_wins.items() if win is False]

    # 6. Check for game-winning states on next move
    self_win_states = {}
    for col in wins:
        assert state[fill_levels[col], col] == 0
        state[fill_levels[col], col] = role
        fill_levels[col] += 1
        opponent_available_moves = game.available_moves(state)
        self_n_wins = {}
        for col_opp in opponent_available_moves:
            assert state[fill_levels[col_opp], col_opp] == 0
            state[fill_levels[col_opp], col_opp] = opponent
            self_wins = _wins_from_next_move(game, role, board_full=bf2)
            self_n_wins[col_opp] = sum(self_wins.values())
            state[fill_levels[col_opp], col_opp] = 0
        fill_levels[col] -= 1
        state[fill_levels[col], col] = 0
        winning_states = {c: n_wins > 0 for c, n_wins in self_n_wins.items()}
        self_win_states[col] = all(winning_states.values())
    self_wins = [c for c, win in self_win_states.items() if win]
    if self_wins:
        return 1.0, self_wins

    # Otherwise, return no value
    return None, list(wins.keys())

In [18]:
game = Connect4()
board_full, state = game._empty_board_state()
state[:] = np.array([
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]
])

role = 1
assert not any(
    _wins_from_next_move(game, role, board_full=board_full).values()
)

# Best starting move
check_for_obvious_move(game, role, board_full=board_full)

(None, [3])

In [19]:
game = Connect4()
board_full, state = game._empty_board_state()
state[:] = np.array([
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]
])

role = 2
assert not any(
    _wins_from_next_move(game, role, board_full=board_full).values()
)

# Best response to best starting move
check_for_obvious_move(game, role, board_full=board_full)

(None, [3])

In [20]:
game = Connect4()
board_full, state = game._empty_board_state()
state[:] = np.array([
    [0, 1, 1, 1, 2, 2, 1],
    [0, 0, 1, 0, 2, 1, 0],
    [0, 0, 0, 0, 2, 2, 0],
    [0, 0, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 1, 0]
])

role = 1
assert _wins_from_next_move(game, role, board_full=board_full) == \
    {0: True, 1: False, 2: False, 3: False, 4: False, 6: False}

role = 2
assert _wins_from_next_move(game, role, board_full=board_full) == \
    {0: False, 1: False, 2: False, 3: False, 4: True, 6: False}

role = 1
assert check_for_obvious_move(game, role, board_full=board_full) == \
    (1.0, [0])

role = 2
assert check_for_obvious_move(game, role, board_full=board_full) == \
    (1.0, [4]) 

In [21]:
game = Connect4()
board_full, state = game._empty_board_state()
state[:] = np.array([
    [0, 2, 1, 1, 2, 2, 1],
    [0, 0, 1, 0, 2, 1, 0],
    [0, 0, 0, 0, 2, 1, 0],
    [0, 0, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 1, 0]
])

role = 1
# Should block win by player 2
assert check_for_obvious_move(game, role, board_full=board_full) == \
    (None, [4])

In [22]:
game = Connect4()
board_full, state = game._empty_board_state()
state[:] = np.array([
    [0, 1, 2, 1, 2, 2, 1],
    [0, 0, 1, 0, 2, 1, 0],
    [0, 0, 0, 0, 2, 1, 0],
    [0, 0, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 1, 0]
])

role = 1
# Can't win but at least block one of moves
assert check_for_obvious_move(game, role, board_full=board_full) == \
    (-1.0, [3, 4])

In [23]:
game = Connect4()
board_full, state = game._empty_board_state()
state[:] = np.array([
    [0, 0, 1, 1, 2, 2, 1],
    [0, 0, 1, 2, 2, 1, 0],
    [0, 0, 0, 0, 2, 2, 0],
    [0, 0, 0, 0, 1, 2, 0],
    [0, 0, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 1, 0]
])

role = 1
assert not any(
    _wins_from_next_move(game, role, board_full=board_full).values()
)
# This should identify the winning move (1)
assert check_for_obvious_move(game, role, board_full=board_full) == \
    (1.0, [1])

In [24]:
game = Connect4()
board_full, state = game._empty_board_state()
state[:] = np.array([
    [0, 0, 1, 1, 0, 0, 0],
    [0, 0, 0, 2, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]
])

role = 2
assert not any(
    _wins_from_next_move(game, role, board_full=board_full).values()
)

# TODO: This should identify a blocking move (1 or 4)
check_for_obvious_move(game, role, board_full=board_full)

(None, [0, 1, 2, 3, 4, 5, 6])

In [25]:
STOP!!

SyntaxError: invalid syntax (<ipython-input-25-480d53eab50e>, line 1)

In [ ]:
rows = np.empty((4, size[1]), dtype='int8')
for i in range(4):
    rows[i] = (fl-i).clip(-1,)
rows

In [ ]:
cols = np.empty((4, size[1]), dtype='int8')
cols[:] = np.arange(1,size[1]+1)
cols

In [ ]:
board_full

In [ ]:
board_full[(rows, cols)]

In [ ]:
m_level = int(np.median(fl))
m_level

In [ ]:
state_key = np.empty((5, size[1]), dtype='int8')
state_key[0, :] = (fl - m_level).clip(-1, 2)
state_key[1:, :] = board_full[(rows, cols)]
state_key

### Find all possible (valid) combinations of states for 4x4 segment

In [ ]:
from itertools import product

combinations = np.array(list(product(range(5), repeat=4))).sum(axis=1)
assert len(combinations) == 625

In [ ]:
n_discs = pd.Series(combinations).value_counts().sort_index().rename('No. of boards')
n_discs.index.name = 'No. of discs'
summary = pd.concat([
    n_discs, 
    pd.Series(2**n_discs.index.values, index=n_discs.index, name='Disc combinations')
], axis=1)
summary['Total combinations'] = summary['No. of boards'] * summary['Disc combinations']
summary

In [ ]:
summary['Total combinations'].sum()

### Generate a compact, hashable state representation

In [ ]:
a1 = np.array([64, 16, 4, 1])
a2 = np.array([16777216, 65536, 256, 1])

def generate_state_key_uint32(grid):
    """Convert 4x4 int8 array to int64 value.
    """
    assert grid.dtype == 'int8'
    assert grid.shape == (4, 4)
    return (np.sum(grid*a1, axis=1)*a2).sum().astype('uint32')

grid = 3*np.ones((4,4), dtype='int8') 
assert generate_state_key_uint32(grid) == 2**32-1
assert generate_state_key_uint32(grid).dtype == 'uint32'
assert (generate_state_key_uint32(grid) + 1).dtype == 'int64'
grid = np.zeros((4,4), dtype='int8') 
assert generate_state_key_uint32(grid) == 0
grid[:, 3] = 1
assert generate_state_key_uint32(grid) == 1 + 256 + 256**2 + 256**3
grid = np.ones((4,3), dtype='int8') 
try:
    generate_state_key_uint32(grid)
except AssertionError:
    pass

grid = np.random.choice([0, 1, 2], size=16).astype('int8').reshape((4,4))
print(grid)
generate_state_key_uint32(grid)

In [ ]:
# sum([30.9, 31, 31.1, 28.9])/4 == 30.5
%timeit generate_state_key_uint32(grid)

In [ ]:
a1 = np.array([6, 4, 2, 0])
a2 = np.array([24, 16, 8, 0])

def generate_state_key_uint32(grid):
    """Convert 4x4 int8 array to int64 value.
    """
    assert grid.dtype == 'int8'
    return np.sum(np.sum(grid << a1, axis=1) << a2).astype('uint32')

grid = 3*np.ones((4,4), dtype='int8') 
assert generate_state_key_uint32(grid) == 2**32-1
assert generate_state_key_uint32(grid).dtype == 'uint32'
assert (generate_state_key_uint32(grid) + 1).dtype == 'int64'
grid = np.zeros((4,4), dtype='int8') 
assert generate_state_key_uint32(grid) == 0
grid[:, 3] = 1
assert generate_state_key_uint32(grid) == 1 + 256 + 256**2 + 256**3
grid = np.ones((4,3), dtype='int8') 
try:
    generate_state_key_uint32(grid)
except ValueError:
    pass

grid = np.random.choice([0, 1, 2], size=16).astype('int8').reshape((4,4))
print(grid)
generate_state_key_uint32(grid)

In [ ]:
# sum([31.2, 31.9, 31.8, 32.4])/4 == 31.8
%timeit generate_state_key_uint32(grid)

In [ ]:
a = np.array([1, 2, 3, 0], dtype='int8')

v = np.sum(a << np.array([6, 4, 2, 0])).astype('uint32')

v

In [ ]:
(v >> 6) & 3, (v >> 4) & 3, (v >> 2) & 3, v & 3

In [ ]:
np.array([v >> 6, v >> 4, v >> 2, v], dtype='int8') & 3